In [1]:
import os
import pandas as pd

#specifying the path 
path = "D:\Python data\Reuters data/"

# Creating the i/o handle 
f = open(path + "training_money-fx_14777.txt")

In [2]:
# for line in f:
#     print(line)

In [3]:
sentences = " ".join([word.strip() for word in f if word != "" ])

sentences

'Bank of Japan buys dollars shortly after opening at around 145.30 yen -- dealers  Bank of Japan buys dollars shortly after opening at around 145.30 yen -- dealers  '

In [4]:
import os

# total no of files in that particular directory
files = os.listdir(path)

# Selecting the files that have ".txt" as their extension
files = [file for file in files if ".txt" in file]

In [5]:
# Now iterating through each and every file and also creating i/o handle as well as seperating the crude and money data
target = []
article = []
for file in files :
    f = open(path + file,encoding="latin-1")
    sentences = " ".join([word.strip() for word in f if word.strip() != " " ])
    
    if "crude" in file:
        target.append("crude")
        article.append(sentences)
    else:
        target.append("money")
        article.append(sentences)
    f.close()

In [6]:
# Creating a dataframe 
mydata = pd.DataFrame({"Target":target,"Article":article})

mydata.head()

,Article,Target
0,CANADA OIL EXPORTS RISE 20 PCT IN 1986 Canadia...,crude
1,BP &lt;BP> DOES NOT PLAN TO HIKE STANDARD &lt;...,crude
2,BP&lt;BP> OFFER RAISES EXPECTATIONS FOR OIL VA...,crude
3,USX &lt;X> SAYS TALKS ENDED WITH BRITISH PETRO...,crude
4,BP &lt;BP> MAY HAVE TO RAISE BID - ANALYSTS B...,crude


In [7]:
import re
import nltk
from nltk.stem.wordnet import WordNetLemmatizer
lemma = WordNetLemmatizer()
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from string import punctuation
mystop = set(stopwords.words("english") + list(punctuation))

In [8]:
corpus = []
for i in range(len(mydata.Article)):
    review = re.sub("[^a-zA-Z]"," ", str(mydata.Article[i]))
    review = review.lower()
    review = re.sub("\s+"," ", review)
    review = review.split()
    review = [lemma.lemmatize(word) for word in review if word not in mystop]
    review = " ".join(review)
    corpus.append(review)

In [9]:
corpus[926]

'u k money market shortage forecast revised bank england said revised forecast shortage money market mln stg taking account morning operation noon bank estimated shortfall mln stg'

In [10]:
#     mydata.Article[i] = [word for word in mydata.Article[i] if word not in mystop]
#     mydata.Article[i] = [lemma.lemmatize(word) for word in mydata.Article[i]]

In [11]:
mydata.head()

,Article,Target
0,CANADA OIL EXPORTS RISE 20 PCT IN 1986 Canadia...,crude
1,BP &lt;BP> DOES NOT PLAN TO HIKE STANDARD &lt;...,crude
2,BP&lt;BP> OFFER RAISES EXPECTATIONS FOR OIL VA...,crude
3,USX &lt;X> SAYS TALKS ENDED WITH BRITISH PETRO...,crude
4,BP &lt;BP> MAY HAVE TO RAISE BID - ANALYSTS B...,crude


In [12]:
mydata["article"] = corpus

In [13]:
del mydata["Article"]

In [14]:
mydata.head()

,Target,article
0,crude,canada oil export rise pct canadian oil export...
1,crude,bp lt bp plan hike standard lt srd bid british...
2,crude,bp lt bp offer raise expectation oil value bri...
3,crude,usx lt x say talk ended british petroleum usx ...
4,crude,bp lt bp may raise bid analyst british petrole...


In [15]:
# mydata = mydata.rename(columns= {"corpus" : "article", "Target":"target"})

# train test split

In [16]:
from sklearn.model_selection import train_test_split

In [17]:
article_train,article_test = train_test_split(mydata , test_size = 0.2, random_state = 42)

In [18]:
article_train.reset_index(drop=True , inplace= True)

In [19]:
article_train.head()

,Target,article
0,money,n dealer believe fed intervened buy dlrs feder...
1,crude,creditor seek swift rescue package japan line ...
2,money,german net currency reserve rise west german n...
3,money,indonesian bank raise interest rate tight mone...
4,money,currency future key g g meeting news agreement...


In [20]:
article_train.shape

(741, 2)

# Creating the Bag of words model 

In [21]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(min_df= 30, max_df=500)

In [22]:
# Doing the transformation of the train data
X = cv.fit_transform(article_train["article"])

independent_train = pd.DataFrame(X.toarray() , columns= cv.get_feature_names())

independent_train.shape

(741, 395)

In [23]:
# Doing the transformation of the test data
Y = cv.transform(article_test["article"])

independent_test = pd.DataFrame(Y.toarray() ,columns= cv.get_feature_names())

independent_test.shape


(186, 395)

In [24]:
# Creating dummies for labels

article_train["Target"] = (article_train["Target"] == "money").astype(int)
article_test["Target"] = (article_test["Target"] == "money").astype(int)

D:\Python\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
D:\Python\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


# Using Naive_bayes for Modeling

In [25]:
from sklearn.naive_bayes import MultinomialNB

nb = MultinomialNB()

In [26]:
# Fitting the model 
model = nb.fit(independent_train , article_train.Target)

prediction = model.predict(independent_test)

## Accuracy score

In [27]:
from sklearn.metrics import accuracy_score

accuracy_score(prediction , article_test.Target)

0.9838709677419355

## Confusion matrix

In [28]:
from sklearn.metrics import confusion_matrix

confusion_matrix(prediction , article_test.Target)

array([[86,  0],
       [ 3, 97]], dtype=int64)

# Using Support Vector Machines for Modeling

In [29]:
from sklearn.svm import SVC

sv = SVC()

In [30]:
# Fitting the model 
svmodel = sv.fit(independent_train , article_train.Target)

prediction = svmodel.predict(independent_test)

## Accuracy score

In [31]:
from sklearn.metrics import accuracy_score

accuracy_score( article_test.Target , prediction )

0.989247311827957

## Confusion matrix

In [32]:
from sklearn.metrics import confusion_matrix

confusion_matrix(article_test.Target, prediction)

array([[87,  2],
       [ 0, 97]], dtype=int64)

## Creating Tf-idf vector

In [33]:
from sklearn.feature_extraction.text import TfidfVectorizer

tv = TfidfVectorizer(min_df= 30, max_df=500)

In [34]:
# Doing the transformation of the train data
A = tv.fit_transform(article_train["article"])

independent_train_tfidf = pd.DataFrame(A.toarray() , columns= tv.get_feature_names())

independent_train_tfidf.head()

,accord,according,account,action,add,added,adding,administration,afternoon,agency,...,west,whether,within,work,world,would,year,yen,yesterday,york
0,0.000000,0.0,0.139766,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,...,0.000000,0.154113,0.0,0.000000,0.000000,0.000000,0.000000,0.226449,0.0,0.0
1,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,...,0.000000,0.000000,0.0,0.072048,0.112176,0.000000,0.038010,0.257454,0.0,0.0
2,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,...,0.127981,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0
3,0.000000,0.0,0.084442,0.0,0.0,0.067771,0.0,0.0,0.0,0.088632,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.302983,0.000000,0.0,0.0
4,0.128314,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.125581,0.052915,0.250889,0.0,0.0


In [35]:
# Doing the transformation of the test data
B = tv.transform(article_test["article"])

independent_test_tfidf = pd.DataFrame(B.toarray() ,columns= tv.get_feature_names())

independent_test_tfidf.shape

(186, 395)

## Performing a fit and prediction on tf-idf using naive-bayes

In [36]:
nb_tfidf_model = nb.fit(independent_train_tfidf, article_train.Target)

prediction_tfidf = nb_tfidf_model.predict(independent_test_tfidf)

## Accuracy score

In [37]:
accuracy_score(article_test.Target , prediction_tfidf)

0.978494623655914

## Confusion matrix

In [38]:
confusion_matrix(article_test.Target , prediction_tfidf)

array([[85,  4],
       [ 0, 97]], dtype=int64)

## Performing a fit and prediction on tf-idf using Support vector machines

In [39]:
svmodel_tfidf = sv.fit(independent_train_tfidf , article_train.Target)

prediction_tfidf = svmodel_tfidf.predict(independent_test_tfidf)

## Accuracy score

In [40]:
accuracy_score(article_test.Target , prediction_tfidf)

0.521505376344086

In [41]:
confusion_matrix(article_test.Target , prediction_tfidf)

array([[ 0, 89],
       [ 0, 97]], dtype=int64)